# Python For Data Analysis
## Class 2

The objectives of this class are for y'all to have:

1. Gained familiarity with `pandas` API
2. Started exploring our 311 data set
3. Learned the split / apply / combine data munging paradigm
4. Learned some

interactive data analysis tricks

In [49]:
import pandas as pd
import numpy as np
import matplotlib
%matplotlib inline

In [3]:
# Load the data
complaints = pd.read_csv('../pandas-cookbook/data/311-service-requests.csv', low_memory=False)

# Note: It's nice to do this in its own cell so we don't ever have to-rerun this costly line

In [4]:
# Let's clean up our data by doing a few things:
# 1) let's limit to a few columns we know are going to be interesting
# 2) let's clean the column names so we don't have to deal with spaces or capital letters

In [5]:
complaints.columns
useful_cols = ['Created Date', 'Closed Date','Due Date', 'Agency Name', 'Complaint Type', 'Borough', 'Status']
cleaned = complaints[useful_cols]

In [6]:
cleaned.head()

,Created Date,Closed Date,Agency Name,Complaint Type,Borough,Status
0,10/31/2013 02:08:41 AM,NaN,New York City Police Department,Noise - Street/Sidewalk,QUEENS,Assigned
1,10/31/2013 02:01:04 AM,NaN,New York City Police Department,Illegal Parking,QUEENS,Open
2,10/31/2013 02:00:24 AM,10/31/2013 02:40:32 AM,New York City Police Department,Noise - Commercial,MANHATTAN,Closed
3,10/31/2013 01:56:23 AM,10/31/2013 02:21:48 AM,New York City Police Department,Noise - Vehicle,MANHATTAN,Closed
4,10/31/2013 01:53:44 AM,NaN,Department of Health and Mental Hygiene,Rodent,MANHATTAN,Pending


Exercise:
* programatically lower-cases the column names and change the spaces to under-scores
  * Try not to rely on the current ordering of the columns to do this

In [10]:
# One solution
cleaned.rename(columns=lambda x: x.lower().replace(' ','_'), inplace=True)

/Users/michael/.virtualenvs/python-for-data-analysis/lib/python3.5/site-packages/pandas/core/frame.py:2834: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)


### Value Counts

In [11]:
cleaned.head()

,created_date,closed_date,agency_name,complaint_type,borough,status
0,10/31/2013 02:08:41 AM,NaN,New York City Police Department,Noise - Street/Sidewalk,QUEENS,Assigned
1,10/31/2013 02:01:04 AM,NaN,New York City Police Department,Illegal Parking,QUEENS,Open
2,10/31/2013 02:00:24 AM,10/31/2013 02:40:32 AM,New York City Police Department,Noise - Commercial,MANHATTAN,Closed
3,10/31/2013 01:56:23 AM,10/31/2013 02:21:48 AM,New York City Police Department,Noise - Vehicle,MANHATTAN,Closed
4,10/31/2013 01:53:44 AM,NaN,Department of Health and Mental Hygiene,Rodent,MANHATTAN,Pending


In [12]:
cleaned.complaint_type.unique()

array(['Noise - Street/Sidewalk', 'Illegal Parking', 'Noise - Commercial',
       'Noise - Vehicle', 'Rodent', 'Blocked Driveway',
       'Noise - House of Worship', 'Street Light Condition',
       'Harboring Bees/Wasps', 'Taxi Complaint', 'Homeless Encampment',
       'Traffic Signal Condition', 'Food Establishment', 'Noise - Park',
       'Broken Muni Meter', 'Benefit Card Replacement',
       'Sanitation Condition', 'ELECTRIC', 'PLUMBING', 'HEATING',
       'GENERAL CONSTRUCTION', 'Street Condition', 'Consumer Complaint',
       'Derelict Vehicles', 'Noise', 'Drinking', 'Indoor Air Quality',
       'Panhandling', 'Derelict Vehicle', 'Lead', 'Water System',
       'Noise - Helicopter', 'Homeless Person Assistance',
       'Root/Sewer/Sidewalk Condition', 'Sidewalk Condition', 'Graffiti',
       'DOF Literature Request', 'Animal in a Park',
       'Overgrown Tree/Branches', 'Air Quality', 'Dirty Conditions',
       'Water Quality', 'Other Enforcement', 'Collection Truck Noise',
     

In [20]:
# Let's figure out what the top complaints are
cleaned['complaint_type'].value_counts()


HEATING                                 14200
GENERAL CONSTRUCTION                     7471
Street Light Condition                   7117
DOF Literature Request                   5797
PLUMBING                                 5373
PAINT - PLASTER                          5149
Blocked Driveway                         4590
NONCONST                                 3998
Street Condition                         3473
Illegal Parking                          3343
Noise                                    3321
Traffic Signal Condition                 3145
Dirty Conditions                         2653
Water System                             2636
Noise - Commercial                       2578
ELECTRIC                                 2350
Broken Muni Meter                        2070
Noise - Street/Sidewalk                  1928
Sanitation Condition                     1824
Rodent                                   1632
Sewer                                    1627
Consumer Complaint                

In [22]:
cleaned['complaint_type_cln'] = cleaned['complaint_type'].str.lower()
# cleaned.is_copy = False 

/Users/michael/.virtualenvs/python-for-data-analysis/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [23]:
cleaned.head()

,created_date,closed_date,agency_name,complaint_type,borough,status,complaint_type_cln
0,10/31/2013 02:08:41 AM,NaN,New York City Police Department,Noise - Street/Sidewalk,QUEENS,Assigned,noise - street/sidewalk
1,10/31/2013 02:01:04 AM,NaN,New York City Police Department,Illegal Parking,QUEENS,Open,illegal parking
2,10/31/2013 02:00:24 AM,10/31/2013 02:40:32 AM,New York City Police Department,Noise - Commercial,MANHATTAN,Closed,noise - commercial
3,10/31/2013 01:56:23 AM,10/31/2013 02:21:48 AM,New York City Police Department,Noise - Vehicle,MANHATTAN,Closed,noise - vehicle
4,10/31/2013 01:53:44 AM,NaN,Department of Health and Mental Hygiene,Rodent,MANHATTAN,Pending,rodent


In [24]:
cleaned['complaint_type_cln'].value_counts()

heating                                 14200
general construction                     7471
street light condition                   7117
dof literature request                   5797
plumbing                                 5439
paint - plaster                          5149
blocked driveway                         4590
nonconst                                 3998
street condition                         3473
illegal parking                          3343
noise                                    3321
traffic signal condition                 3145
dirty conditions                         2653
water system                             2636
noise - commercial                       2578
electric                                 2350
broken muni meter                        2070
noise - street/sidewalk                  1928
sanitation condition                     1824
rodent                                   1632
sewer                                    1627
taxi complaint                    

### Replacing data

In [59]:
# which rows have rats and noisy vehicles?
cleaned["complaint_type"].isin(['Rodent', 'Noise - Vehicle'])

0         False
1         False
2         False
3          True
4          True
5         False
6         False
7         False
8         False
9         False
10        False
11        False
12        False
13         True
14         True
15        False
16        False
17        False
18        False
19        False
20        False
21        False
22         True
23        False
24        False
25        False
26        False
27        False
28        False
29        False
          ...  
111039    False
111040    False
111041    False
111042    False
111043    False
111044    False
111045    False
111046    False
111047    False
111048    False
111049    False
111050    False
111051    False
111052    False
111053    False
111054    False
111055    False
111056    False
111057    False
111058    False
111059    False
111060    False
111061    False
111062    False
111063    False
111064    False
111065    False
111066    False
111067    False
111068    False
Name: complaint_type, dt

In [77]:
# Replace some values
mask = cleaned["complaint_type"].isin(['Rodent', 'Noise - Vehicle'])
new_series = cleaned['complaint_type']
# new_series = cleaned['complaint_type'].copy()
new_series[mask] = 'rats or cars'



In [78]:
new_series.head()

0    Noise - Street/Sidewalk
1            Illegal Parking
2         Noise - Commercial
3               rats or cars
4               rats or cars
Name: complaint_type, dtype: object

Exercise:
* Write a function that takes a column name, a number n, and a dataframe as an argument, and returns a column with the top n categories and all other categories as "other"

### Time to resolution

In [34]:
### Creating a column with time-to-close
### pd.options.mode.chained_assignment = None
cleaned.dtypes
cleaned['created_date'] = pd.to_datetime(cleaned['created_date'])
cleaned['closed_date'] = pd.to_datetime(cleaned['closed_date'])
cleaned.head()

,created_date,closed_date,agency_name,complaint_type,borough,status,complaint_type_cln,time_to_resolution
0,2013-10-31 02:08:41,NaT,New York City Police Department,Noise - Street/Sidewalk,QUEENS,Assigned,noise - street/sidewalk,NaT
1,2013-10-31 02:01:04,NaT,New York City Police Department,Illegal Parking,QUEENS,Open,illegal parking,NaT
2,2013-10-31 02:00:24,2013-10-31 02:40:32,New York City Police Department,Noise - Commercial,MANHATTAN,Closed,noise - commercial,00:40:08
3,2013-10-31 01:56:23,2013-10-31 02:21:48,New York City Police Department,Noise - Vehicle,MANHATTAN,Closed,noise - vehicle,00:25:25
4,2013-10-31 01:53:44,NaT,Department of Health and Mental Hygiene,Rodent,MANHATTAN,Pending,rodent,NaT


In [47]:
cleaned.dtypes

created_date           datetime64[ns]
closed_date            datetime64[ns]
agency_name                    object
complaint_type                 object
borough                        object
status                         object
complaint_type_cln             object
time_to_resolution    timedelta64[ns]
dtype: object

In [52]:
cleaned['time_to_resolution'] = (cleaned['closed_date'] - cleaned['created_date']) / np.timedelta64(1, 'm')
cleaned.head(20)

,created_date,closed_date,agency_name,complaint_type,borough,status,complaint_type_cln,time_to_resolution
0,2013-10-31 02:08:41,NaT,New York City Police Department,Noise - Street/Sidewalk,QUEENS,Assigned,noise - street/sidewalk,NaN
1,2013-10-31 02:01:04,NaT,New York City Police Department,Illegal Parking,QUEENS,Open,illegal parking,NaN
2,2013-10-31 02:00:24,2013-10-31 02:40:32,New York City Police Department,Noise - Commercial,MANHATTAN,Closed,noise - commercial,40.133333
3,2013-10-31 01:56:23,2013-10-31 02:21:48,New York City Police Department,Noise - Vehicle,MANHATTAN,Closed,noise - vehicle,25.416667
4,2013-10-31 01:53:44,NaT,Department of Health and Mental Hygiene,Rodent,MANHATTAN,Pending,rodent,NaN
5,2013-10-31 01:46:52,NaT,New York City Police Department,Noise - Commercial,QUEENS,Open,noise - commercial,NaN
6,2013-10-31 01:46:40,NaT,New York City Police Department,Blocked Driveway,QUEENS,Assigned,blocked driveway,NaN
7,2013-10-31 01:44:19,2013-10-31 01:58:49,New York City Police Department,Noise - Commercial,QUEENS,Closed,noise - commercial,14.500000
8,2013-10-31 01:44:14,2013-10-31 02:28:04,New York City Police Department,Noise - Commercial,MANHATTAN,Closed,noise - commercial,43.833333
9,2013-10-31 01:34:41,2013-10-31 02:23:51,New York City Police Department,Noise - Commercial,BROOKLYN,Closed,noise - commercial,49.166667


### Group By and Aggregate Metrics

In [53]:
# group our data by complaint type
by_complaint = cleaned.groupby('complaint_type_cln')

In [54]:
# average response time
by_complaint['time_to_resolution'].mean()

complaint_type_cln
adopt-a-basket                                 1446.000000
agency issues                                          NaN
air quality                                    1516.666667
animal abuse                                    223.147866
animal facility - no permit                            NaN
animal in a park                               3260.302000
appliance                                      3730.909091
asbestos                                       1542.030556
beach/pool/sauna complaint                    28063.312500
benefit card replacement                          0.602283
best/site safety                                346.853667
bike rack condition                            8601.200000
bike/roller/skate chronic                       126.328646
blocked driveway                                179.920552
boilers                                                NaN
bridge condition                               1376.686842
broken muni meter                    

In [59]:
###cleaned.loc[cleaned['time_to_resolution']<0,:].head()

Exercise:
  * What's going on with negative time-to-resolution?
  * Determine which types of complaints are most often late (closed_date > due_date)
  * From which boroughs?


ToDo:
* Introduce summary stats with group by
  * Reset_index to convert to data frame
  * Introduce common aggregates
  * Exercise: find the most common hour for a complaint by borough
